**ME700 HW3 Part A**  
implement an example where you validate the performance of the code with an analytical solution.

Problem: I used the problem presented in [the 2D linear elasticity tutorial in Fenics](https://comet-fenics.readthedocs.io/en/latest/demo/elasticity/2D_elasticity.py.html). All the parameters are taken from this example. A 2D rectangular beam is fixed at one end and a distributed force is applied to that beam. In small displacements, the deflection of the tip of the beam can be approximated analytically. Try to compare the analytic and numerical results.   

PS: It should be noted that the definition of force in this problem is for a distributed traction force, while in the example shown in fenics, it's for a body force. But since we are using a slender 2D beam, it is safe to think that the analytical solutions for these two are the same.

**Importing Modules**

In [1]:
# importing modules
from finiteelementanalysis import pre_process as pre
from finiteelementanalysis import pre_process_demo_helper_fcns as pre_demo
from finiteelementanalysis.solver import hyperelastic_solver
from finiteelementanalysis import discretization as di
import numpy as np
import matplotlib.pyplot as plt

**Functions**

In [ ]:
def solve_hyperelastic_rect_mesh(ele_type, L_x, L_y, N_x, N_y, q, nr_num_steps, material_properties_dict):
    """Solves a 2D hyperelastic problem on a rectangular mesh."""

    #--- Get Element Info ---
    _, ndof, _ = di.element_info(ele_type)

    #--- Generate Mesh ---
    origin_x, origin_y = 0.0, 0.0
    coords, connect = pre.generate_rect_mesh_2d(
        ele_type,
        origin_x, origin_y,
        origin_x + L_x, origin_y + L_y,
        N_x, N_y
    )

    #--- Define Boundaries ---
    boundary_nodes, boundary_edges = pre.identify_rect_boundaries(
        coords, connect, ele_type,
        origin_x, origin_x + L_x,
        origin_y, origin_y + L_y
    )

    # Fixed boundary on the left edge: u_x = u_y = 0
    fixed_nodes = pre.assign_fixed_nodes_rect(boundary_nodes, "left", 0.0, 0.0)

    # Apply uniform load on the top edge in x-direction
    dload_info = pre.assign_uniform_load_rect(boundary_edges, "top", 0.0, q)

    #--- Material Properties ---
    mu = material_properties_dict['mu']
    kappa = material_properties_dict['kappa']

    material_props = np.array([mu, kappa])

    #--- Solve Nonlinear Problem ---
    displacements_all, nr_info_all = hyperelastic_solver(
        material_props,
        ele_type,
        coords.T,
        connect.T,
        fixed_nodes,
        dload_info,
        nr_print=True,
        nr_num_steps=nr_num_steps,
        nr_tol=1e-9,
        nr_maxit=30
    )

    return coords, connect, displacements_all, nr_info_all

**Parameters**

In [3]:
#--- Material Properties ---
nu = 0.3
E = 100000

mu = E / (2.0 * (1.0 + nu))
kappa = E / (3.0 * (1.0 - 2.0 * nu))

material_properties_dict = {'nu': nu, 'E': E, 'mu': mu, 'kappa': kappa}

#--- Geometry Parameters ---
ratio = 25
L_y = 1                          # Width
L_x = ratio * L_y               # Length

#--- Mesh Sizes: (N_x, N_y) ---
ele_type = 'D2_nn8_quad'        # Type of element for the mesh
mesh_size = (10, 250)
N_x, N_y = mesh_size

**Solving**

In [4]:
#--- Solver Setup ---
q = -0.001                       # Applied traction in x-direction
nr_num_steps = 1               # Number of Newton-Raphson steps

#--- Run Simulation and Collect Results ---
coords, connect, displacements_all, _ = solve_hyperelastic_rect_mesh(
    ele_type, L_x, L_y, N_x, N_y, q, nr_num_steps, material_properties_dict
)

Step 0, load factor = 1.000
Iteration 1, Correction=1.000000e+00, Residual=3.645553e-07, tolerance=1.000000e-09
Iteration 2, Correction=1.134087e-04, Residual=2.895151e-06, tolerance=1.000000e-09
Iteration 3, Correction=1.786909e-09, Residual=3.067995e-13, tolerance=1.000000e-09
Iteration 4, Correction=6.519525e-14, Residual=2.974270e-13, tolerance=1.000000e-09


**Post Processing the Results**

In [5]:
dispfield_final = displacements_all[-1].reshape(-1, 2)
tol = 1e-8
# Find the node at the tip center (x = L_x, y = L_y / 2)
ii = np.where(
    (np.abs(coords[:, 0] - L_x) < tol) & 
    (np.abs(coords[:, 1] - L_y / 2) < tol)
)

tipcenterdisp_numerical = dispfield_final[ii, 1].item()
print( 'Numerical tip displacement is', tipcenterdisp_numerical )

Numerical tip displacement is -0.005288135751410938


**Analytical vs Numerical**

In [ ]:
# Analytical Solution
I = L_y ** 4 / 12
E_effective = E / ( 1 - nu ** 2 )
tipcenterdisp_theory = q * L_x**4 / ( 8 * E * I )
tipcenterdisp_theory_corrected = q * L_x**4 / ( 8 * E_effective * I )

error_percent = 100 * np.abs( ( tipcenterdisp_theory - tipcenterdisp_numerical ) / tipcenterdisp_theory )
error_percent_corrected = 100 * np.abs( ( tipcenterdisp_theory_corrected - tipcenterdisp_numerical ) / tipcenterdisp_theory_corrected )
#------------------
print(f'Numerical displacement of the tip of the beam is: {tipcenterdisp_numerical:.6f} ')
print(f'Analytical displacement of the tip of the beam is: {tipcenterdisp_theory:.6f} ( {error_percent:.2f}% error )')
print(f"Analytical displacement of the tip of the beam, with the corrected Young's modulus is: {tipcenterdisp_theory_corrected:0.6f} ( {error_percent_corrected:.2f}% error ) ")

Numerical displacement of the tip of the beam is: -0.005288 
Analytical displacement of the tip of the beam is: -0.005859 ( 9.75% error )
Analytical displacement of the tip of the beam, with the corrected Young's modulus is: -0.005332 ( 0.82% error ) 
